In [1]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model('weights_epoch_03_val_loss_0.0174.h5')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 50)]                 0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 50)]                 0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 50, 256)              7155404   ['input_5[0][0]']             
                                                          8                                       
                                                                                                  
 embedding_5 (Embedding)     (None, 50, 256)              2338892   ['input_6[0][0]']       

In [2]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png', show_shapes=True)


In [3]:
!pip install pydot

In [4]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png', show_shapes=True)


In [5]:
for layer in model.layers:
    print(layer.name, layer.trainable)

input_5 True
input_6 True
embedding_4 True
embedding_5 True
lstm_4 True
lstm_5 True
dense_2 True


In [6]:
!pip install matplotlib

In [7]:
from tensorflow import keras

def visualize_model(model):
    print("Model Summary:")
    for i, layer in enumerate(model.layers):
        print(f"Layer {i + 1}: {layer.name} - {layer.__class__.__name__}")
        print(f"  Input shape: {layer.input_shape}")
        print(f"  Output shape: {layer.output_shape}\n")

visualize_model(model)



Model Summary:
Layer 1: input_5 - InputLayer
  Input shape: [(None, 50)]
  Output shape: [(None, 50)]

Layer 2: input_6 - InputLayer
  Input shape: [(None, 50)]
  Output shape: [(None, 50)]

Layer 3: embedding_4 - Embedding
  Input shape: (None, 50)
  Output shape: (None, 50, 256)

Layer 4: embedding_5 - Embedding
  Input shape: (None, 50)
  Output shape: (None, 50, 256)

Layer 5: lstm_4 - LSTM
  Input shape: (None, 50, 256)
  Output shape: [(None, 512), (None, 512), (None, 512)]

Layer 6: lstm_5 - LSTM
  Input shape: [(None, 50, 256), (None, 512), (None, 512)]
  Output shape: [(None, 50, 512), (None, 512), (None, 512)]

Layer 7: dense_2 - Dense
  Input shape: (None, 50, 512)
  Output shape: (None, 50, 91363)



In [8]:
# !pip install graphviz

In [9]:
# !pip install pydot
# !pip install graphviz


In [10]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [17]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle
import pandas as pd

df = pd.read_csv('cleaned_data.csv')

questions = df['question'].astype(str).tolist()
responses = df['response'].astype(str).tolist()

# Create and fit the tokenizers for both questions and responses
tokenizer_questions = Tokenizer()
tokenizer_questions.fit_on_texts(questions)

tokenizer_responses = Tokenizer()
tokenizer_responses.fit_on_texts(responses)


with open('tokenizer_questions.pkl', 'wb') as handle:
    pickle.dump(tokenizer_questions, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('tokenizer_responses.pkl', 'wb') as handle:
    pickle.dump(tokenizer_responses, handle, protocol=pickle.HIGHEST_PROTOCOL)


def generate_response(input_question):
    if not isinstance(input_question, str):
        input_question = str(input_question)

    input_question_seq = tokenizer_questions.texts_to_sequences([input_question])

    input_question_seq = pad_sequences(input_question_seq, maxlen=50, padding='post')

    response_seq = model.predict([input_question_seq, input_question_seq])
    generated_sequence = response_seq[0][-1]
    generated_text = tokenizer_responses.sequences_to_texts([generated_sequence])[0]
    
    return generated_text


input_inquiry = "fix your app, its not working."
generated_response = generate_response(input_inquiry)

print("Generated Response:", generated_response)


def generate_response(input_question):
    if not isinstance(input_question, str):
        input_question = str(input_question)

    input_question_seq = tokenizer_questions.texts_to_sequences([input_question])
    
    input_question_seq = pad_sequences(input_question_seq, maxlen=50, padding='post')
    
  
    response_seq = model.predict([input_question_seq, input_question_seq])
    
    print("Tokenized Input Sequence:", input_question_seq)
    print("Generated Sequence:", response_seq)
    
    generated_sequence = response_seq[0][-1]
    generated_token_index = np.argmax(generated_sequence) 
    generated_text = tokenizer_responses.index_word.get(generated_token_index, '')
    print("Generated Text:", generated_text)
    
    return generated_text


input_inquiry2 = "fix your app, its not working."
generated_response = generate_response(input_inquiry2)


print("Generated Response:", generated_response)




1/1 [==============================] - 1s 1s/step
Generated Response: you
1/1 [==============================] - 0s 293ms/step
Tokenized Input Sequence: [[ 93  20  80  60  11 144   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
Generated Sequence: [[[2.7816264e-13 6.3966150e-13 2.8501461e-12 ... 7.5825034e-13
   4.9068718e-12 6.3148820e-14]
  [2.7239155e-17 3.5791847e-15 1.1291669e-13 ... 6.3926216e-15
   1.7263965e-14 1.2136381e-15]
  [6.9483291e-17 3.4916574e-15 5.1542864e-11 ... 1.4603557e-15
   1.4194977e-14 1.8914338e-16]
  ...
  [1.0000000e+00 5.9444372e-14 5.3488043e-21 ... 1.9230209e-13
   6.4003813e-15 7.7258722e-15]
  [1.0000000e+00 5.9440747e-14 5.3486411e-21 ... 1.9229330e-13
   6.4004787e-15 7.7252827e-15]
  [1.0000000e+00 5.9440747e-14 5.3481512e-21 ... 1.9228743e-13
   6.4004787e-15 7.7249295e-15]]]
Generated Text: 
Generated Response: 
